# Feature engineering
## Examining the data
First, we'll take a look at the data that we're working with

In [1]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


So we have 1460 samples, with 81 columns. The goal is to predict the last column, `SalePrice`.

## Splitting Numerical and Categorical features

In [2]:
# first layer - split by datatype
numerical_df = df.select_dtypes(exclude='object')
categorical_df = df.select_dtypes(include='object')
numerical_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [3]:
categorical_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


### Extracting categorical features from numerical dataframe
There are many features in the numerical dataset that are actually categorical by nature, like `YearBuilt`. We will manually examine and justify each feature we deem categorical in this set, and perform one-hot encoding.

In [4]:
numerical_df.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

- `Id`: this is a useless feature, as a house's internal identifying number doesn't determine its price -- this will later be removed.
- `MSSubClass`: this is an identifier for the type of dwelling involved in the sale. Since it's an identifier, it should be a categorical feature.
- `YearBuilt`: this identifies the year the house was built; it should be a categorical feature.
- `YearRemodAdd`: same as `YearRebuilt`.
- `BsmtFullBath`: because this is a binary feature (1 or 0), it's already technically one-hot encoded.
- `BsmtHalfBath`: same as `BsmtFullBath`.
- `GarageYrBlt`: this identifies the year the garage was built -- should be categorical.
- `MoSold`: this identifies the month the house was sold -- should be categorical.
- `YrSold`: same as MoSold, except with year instead of month.

In [5]:
# drop useless features
numerical_df.drop('Id', axis=1, inplace=True)

# drop categorical features
extracted_cat_features = ['MSSubClass', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 
                          'BsmtHalfBath', 'GarageYrBlt', 'MoSold', 'YrSold']
numerical_df.drop(extracted_cat_features, axis=1, inplace=True)

numerical_df.head()

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,2,548,0,61,0,0,0,0,0,208500
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,2,460,298,0,0,0,0,0,0,181500
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,2,608,0,42,0,0,0,0,0,223500
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,3,642,0,35,272,0,0,0,0,140000
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,3,836,192,84,0,0,0,0,0,250000


In [6]:
# add dropped categorical features to categorical df
categorical_df = pd.concat([categorical_df, df[extracted_cat_features]], axis=1)
categorical_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,SaleType,SaleCondition,MSSubClass,YearBuilt,YearRemodAdd,BsmtFullBath,BsmtHalfBath,GarageYrBlt,MoSold,YrSold
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,WD,Normal,60,2003,2003,1,0,2003.0,2,2008
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,WD,Normal,20,1976,1976,0,1,1976.0,5,2007
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,WD,Normal,60,2001,2002,1,0,2001.0,9,2008
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,WD,Abnorml,70,1915,1970,1,0,1998.0,2,2006
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,WD,Normal,60,2000,2000,1,0,2000.0,12,2008


### Extracting numerical features from categorical data
Some data is categorical, but an inherent order is implied; such as small, medium, and large. This type of feature engineering is called *ordinal encoding*.

In [7]:
categorical_df.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'MSSubClass', 'YearBuilt', 'YearRemodAdd',
       'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'MoSold', 'YrSold'],
      dtype='object')

Here, we identify the categories that are inherently ordinal, and should be converted.
- `LandSlope`
- `BsmtQual`
- `BsmtCond`
- `BsmtFinType1`
- `BsmtFinType2`
- `HeatingQC`
- `Electrical`
- `KitchenQual`
- `Functional`
- `FireplaceQu`
- `GarageQual`
- `GarageCond`
- `PoolQC`
One caveat is that for some of these like `GarageQual`, while the data is inherently ordinal, there are also values like `NaN` for homes without garages. So a home without a garage may not necessarily be worse than a home with a bad quality garage.

In [8]:
extracted_ord_features = ['LandSlope', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 
                          'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
                          'GarageQual', 'GarageCond', 'PoolQC']
categorical_df.drop(extracted_ord_features, axis=1, inplace=True)
categorical_df.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,Neighborhood,Condition1,Condition2,...,SaleType,SaleCondition,MSSubClass,YearBuilt,YearRemodAdd,BsmtFullBath,BsmtHalfBath,GarageYrBlt,MoSold,YrSold
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2003,2003,1,0,2003.0,2,2008
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Veenker,Feedr,Norm,...,WD,Normal,20,1976,1976,0,1,1976.0,5,2007
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,CollgCr,Norm,Norm,...,WD,Normal,60,2001,2002,1,0,2001.0,9,2008
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Crawfor,Norm,Norm,...,WD,Abnorml,70,1915,1970,1,0,1998.0,2,2006
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,NoRidge,Norm,Norm,...,WD,Normal,60,2000,2000,1,0,2000.0,12,2008


### Convert ordinal features

In [9]:
extracted_ord_features

['LandSlope',
 'BsmtQual',
 'BsmtCond',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC']